## Step1: initialization, import necessary packages
- files are mounted from local file system "volgenmodel-docker/data" into the docker container path "/home/data" (see "docker-compose.yml")

In [ ]:
%pip install tqdm
import os
import sys
sys.path.append('/home/data')
from os.path import join
from functions import *
import tifffile
import SimpleITK as sitk
import numpy as np
import tifffile
import nibabel as nib
import numpy as np
from tqdm.auto import tqdm

os.getcwd()
os.chdir("/volgenmodel-nipype")
print(f"I'm working in folder: {os.getcwd()}")

## folder initialization
input_folder   = '/home/data/masked_brains'
output_folder  = '/home/data/output_templates'
process_folder = '/home/data/converted_brains'
tmp_folder     = '/home/data/tmp'
result_folder  = '/volgenmodel-nipype'

## parameter init
parameters={'fit_stages': 'lin,1,2', ## for testing 'lin,1,2' for best quality use larger numbers like 'lin,1,1,2,2,3,3,4,5,6' from https://doi.org/10.1016/j.ymeth.2015.01.005
            'ncpus': 40, ## how many cores should the template creation use? be careful, don't use to many or the system could crash
            'img_thresholds': len(fileList(input_folder))*[1], ## the threshold value should be the smallest grey value that is located at one of the edges of the image. 
            'padding_size': 25, ## Padding-Size should be ~10% of the largest dimension (88x150x249 -> paddingSize = 25)
            'min_number_of_brains_in_template': 8
                  }

## Step2: N4 (identical to N3) bias field correction 
- https://simpleitk.readthedocs.io/en/master/link_N4BiasFieldCorrection_docs.html


In [ ]:
files = fileList(input_folder)
files = [join(input_folder,file) for file in files 
         if file.find("_N4") == -1]
out_path = input_folder

for _, mri_file in tqdm(enumerate(files)):
    outputImagePath = join(input_folder, mri_file.split("/")[-1].split(".")[0] + '_N4.nii')
    print(outputImagePath)
    create_N4_img(mri_file, outputImagePath)

## Step 3: cropping based on defined threshold or auto-cropping (if available)
- to find a proper threshold load the image into imageJ and find the lowest grey-value on the border of the tissue (the croping will be done in a cubic shape from the outside)
- you can try to let the function find the threshold by themself by setting: "thresholds='auto'" (TODO: test the auto threshold function!)

In [ ]:
files = fileList(input_folder)
files = [join(input_folder,file) for file in files 
         if file.endswith("nii") and file.find("_N4.") != -1]
out_path = input_folder

for i, mri_file in tqdm(enumerate(files)):    
    outputImagePath = join(input_folder, mri_file.split("/")[-1].split(".")[0] + '_auto-crop.nii')
    """
    create_cropped_imgs(input_folder, 
                       input_folder, 
                       thresholds='auto', 
                       save_as_dtype=np.int16)
    """
    create_cropped_imgs(mri_file, 
                       outputImagePath, 
                       threshold=parameters['img_thresholds'][i], 
                       save_as_dtype=np.int32)

## Step 4: Zero-Padding
- Padding-Size should be ~10% of the largest dimension (88x150x249 -> paddingSize = 25)

In [ ]:
files = fileList(input_folder)
files = [join(input_folder,file) for file in files 
         if file.endswith("nii") and file.find("_N4_auto-crop.") != -1]
out_path = input_folder

for _, mri_file in tqdm(enumerate(files)):
    outputImagePath = join(input_folder, mri_file.split("/")[-1].split(".")[0] + "_padded.nii")
    create_padding_imgs(mri_file, outputImagePath, init_paddingSize='10%')


## Step 5: Nifti to Minc convert
- it also delete possible unused mnc files in the processed folder to prevent issues

In [ ]:
os.system("nii2mnc -help")

In [ ]:
nii_to_minc(input_folder, process_folder)
delete_files(process_folder, 'mnc') ## delete unused .nii files in processed folder

## Step 6: Create the Templates
- set up the create_parameters if necessary
- use only a small version of the fit_stages (lin,1,2) to prevent exessive calculation time
- for the best version of the template you should use the parameterset: "lin,1,2,3,4,5,6,7,8"

In [ ]:
parameters

In [ ]:
delete_files(tmp_folder, 'mnc') ## delete unused .nii files in processed folder
delete_files(tmp_folder, 'nii') ## delete unused .nii files in processed folder
process_files(process_folder, output_folder, tmp_folder, parameters, result_folder)

## Step X: Remove the workflow temp or output files to restart the calculation from scratch or to save space
- will not delete the files copied to the volume outside the docker container!

In [ ]:
#delete_workflow_temp(search_folder="/volgenmodel-nipype/")
#os.system('rm /volgenmodel-nipype/workflow_output_workflowMultiProc50/ -rf')

## Additional Test, check out the format of the example files from: "volgenmodel-fast-example" 
- from: https://github.com/carlohamalainen/volgenmodel-fast-example
- to load them with Fiji you have to convert em to Nifti

In [ ]:
##artifical test with test_examples (so they have to convert to nii to do the whole pipeline)
"""
testlist = os.listdir(input_folder)
for t in testlist:
    it = join(input_folder,t)
    cmd = " ".join(["mnc2nii",it, join(input_folder,t.replace("mnc","nii"))])
    os.system(cmd)
"""

In [ ]:
import os
import nibabel as nib
import numpy as np
from itertools import combinations

def find_nii_files(folder_path):
    """
    Searches for all NIfTI (.nii or .nii.gz) files in a given folder.

    Args:
        folder_path (str): The path to the folder to search.

    Returns:
        list: A list of full paths to the NIfTI files found.
    """
    nii_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith((".nii", ".nii.gz")):
                nii_files.append(os.path.join(root, file))
    return nii_files

def process_nii_combinations(nii_file_list, output_folder):
    """
    Creates combinations of 2 NIfTI files from a list, calculates their
    absolute difference, and saves the result as a new NIfTI file.

    Args:
        nii_file_list (list): A list of full paths to NIfTI files.
        output_folder (str): The path to the folder where the difference
                             NIfTI files will be saved.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    processed_combinations = set()

    for file1_path, file2_path in combinations(nii_file_list, 2):
        # Create a unique identifier for the combination to avoid duplicates
        # based on order (e.g., file1_file2 is same as file2_file1)
        file1_name = os.path.basename(file1_path).replace(".nii", "").replace(".gz", "")
        file2_name = os.path.basename(file2_path).replace(".nii", "").replace(".gz", "")

        # Sort names to create a consistent key for checking processed combinations
        sorted_names = tuple(sorted((file1_name, file2_name)))

        if sorted_names in processed_combinations:
            continue  # Skip if this combination (regardless of order) has already been processed

        print(f"Processing combination: {file1_name} and {file2_name}")

        try:
            # Load the NIfTI files
            img1 = nib.load(file1_path)
            img2 = nib.load(file2_path)

            data1 = img1.get_fdata()
            data2 = img2.get_fdata()

            # Ensure both images have the same dimensions
            if data1.shape != data2.shape:
                print(f"Warning: Dimensions mismatch for {file1_name} ({data1.shape}) and {file2_name} ({data2.shape}). Skipping this combination.")
                continue

            # Calculate the absolute difference
            diff_data = np.abs(data1 - data2)

            # Create a new NIfTI image with the difference data
            # Use the affine from one of the original images
            diff_img = nib.Nifti1Image(diff_data, img1.affine, img1.header)

            # Define the output filename
            output_filename = f"{file1_name}_{file2_name}_diff.nii.gz" # Using .nii.gz for compression
            output_filepath = os.path.join(output_folder, output_filename)

            # Save the new NIfTI file
            nib.save(diff_img, output_filepath)
            print(f"Saved difference to: {output_filepath}")

            processed_combinations.add(sorted_names)

        except Exception as e:
            print(f"Error processing combination {file1_name} and {file2_name}: {e}")

def compare_nii_files(file_path1, file_path2):
    """
    Compares the voxel data content of two NIfTI files.

    Args:
        file_path1 (str): The full path to the first NIfTI file.
        file_path2 (str): The full path to the second NIfTI file.

    Returns:
        int: 1 if the voxel data content of the two files is identical,
             0 otherwise (including if files cannot be loaded or dimensions differ).
    """
    try:
        # Load the NIfTI files
        img1 = nib.load(file_path1)
        img2 = nib.load(file_path2)

        data1 = img1.get_fdata()
        data2 = img2.get_fdata()

        # Compare the data arrays
        if np.array_equal(data1, data2):
            return 1  # Identical
        else:
            return 0  # Not identical

    except Exception as e:
        print(f"Error comparing files {file_path1} and {file_path2}: {e}")
        return 0 # Return 0 if there's an error (e.g., file not found, invalid NIfTI)

# Example Usage:
if __name__ == "__main__":
    # Create a dummy folder structure and dummy .nii files for demonstration
    output_folder  = '/home/data/output_templates'
    print(f"Searching for NIfTI files in: {output_folder}")
    found_nii_files = find_nii_files(output_folder)
    print(f"Found {len(found_nii_files)} NIfTI files:")
    for f in found_nii_files:
        print(f"- {f}")

    if found_nii_files:
        print(f"\nProcessing combinations and saving results to: {output_folder}")
        #process_nii_combinations(found_nii_files, dummy_output_folder)
        print("\nProcessing complete.")

        print("\n--- Comparing NIfTI files ---")
        # Example comparisons using the new function
        print(f"Comparing {os.path.basename(file_A_path)} and {os.path.basename(file_B_path)}: {compare_nii_files(file_A_path, file_B_path)}") # Should be 0 (non-identical)
        print(f"Comparing {os.path.basename(file_A_path)} and {os.path.basename(file_C_path)}: {compare_nii_files(file_A_path, file_C_path)}") # Should be 1 (identical)
        print(f"Comparing {os.path.basename(file_B_path)} and {os.path.basename(file_C_path)}: {compare_nii_files(file_B_path, file_C_path)}") # Should be 0 (non-identical)
        print(f"Comparing {os.path.basename(file_A_path)} and {os.path.basename(file_D_path)}: {compare_nii_files(file_A_path, file_D_path)}") # Should be 0 (different dimensions/error)

    else:
        print("No NIfTI files found to process.")

    # Clean up dummy files (optional)
    # import shutil
    # if os.path.exists(dummy_input_folder):
    #     shutil.rmtree(dummy_input_folder)
    # if os.path.exists(dummy_output_folder):
    #     shutil.rmtree(dummy_output_folder)

In [ ]:
for file_A_path, file_B_path in combinations(found_nii_files, 2):
    print(f"Comparing {os.path.basename(file_A_path)} and {os.path.basename(file_B_path)}: {compare_nii_files(file_A_path, file_B_path)}") # Should be 0 (non-identical)